In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [3]:
!pip install pyarrow

### Q1. Downloading the data

In [15]:
df_jan = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [38]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,duration_min
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50,10.833333


In [16]:
len(df_jan.columns)

19

Re: The data from January has 19 columns

### Q2. Computing duration

In [17]:
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]

In [18]:
df_jan["duration"].std().seconds / 60

42.583333333333336

Re: The standard deviation of the trips duration in January is 42.59 minutes

### Q3. Dropping outliers

In [19]:
df_jan["duration_min"] = df_jan["duration"].map(lambda timestamp: timestamp.seconds / 60)
df_jan["duration_min"]

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration_min, Length: 3066766, dtype: float64

In [29]:
len(df_jan[~df_jan["duration_min"].between(1, 60, inclusive="both")])

57590

In [30]:
df_jan_without_duration_outliers = df_jan.drop(df_jan[~df_jan["duration_min"].between(1, 60, inclusive="both")].index)
len(df_jan_without_duration_outliers)

3009176

In [31]:
len(df_jan_without_duration_outliers) / len(df_jan)

0.9812212604417814

Re: The fraction of records left is 98%

### Q4. One-hot encoding

In [39]:
import re

In [40]:
df_jan_without_duration_outliers.filter(regex=re.compile("pickup|dropoff|loc", re.IGNORECASE))

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID
0,2023-01-01 00:32:10,2023-01-01 00:40:36,161,141
1,2023-01-01 00:55:08,2023-01-01 01:01:27,43,237
2,2023-01-01 00:25:04,2023-01-01 00:37:49,48,238
3,2023-01-01 00:03:48,2023-01-01 00:13:25,138,7
4,2023-01-01 00:10:29,2023-01-01 00:21:19,107,79
...,...,...,...,...
3066761,2023-01-31 23:58:34,2023-02-01 00:12:33,107,48
3066762,2023-01-31 23:31:09,2023-01-31 23:50:36,112,75
3066763,2023-01-31 23:01:05,2023-01-31 23:25:36,114,239
3066764,2023-01-31 23:40:00,2023-01-31 23:53:00,230,79


In [41]:
df_jan_without_duration_outliers["PULocationID"].value_counts()

237    147082
132    144529
236    137402
161    134047
186    108346
        ...  
30          2
58          2
109         1
221         1
245         1
Name: PULocationID, Length: 255, dtype: int64